# Attention Is All You Need


* URL: https://arxiv.org/abs/1706.03762
* authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
* Submitted on 12 Jun 2017 (v1)

## 1. どんなもの？
Transformerの提案。

## 2. 先行研究と比べるとどこが凄い？  
RNNやLSTM、GRUなどの時系列系のモデルは平行して訓練させることができなかった。そこでRNNやCNNを用いずにattention mechanismを利用することで計算を並列化し、時間を短縮させた。さらに精度も上昇させた。

## 3. 技術や手法のキモはどこ？



### Encoder and Decoder Stacks
それぞれ6層のstackから成る。
* Encoder  
  さらに二つのsub layerから成り、residual connectionがある。
  * a multi-head self-attention mechanism
  * positional-wise fully connected feed-forward network  
それぞれのsub_layerのアウトプットはlayer normalizationを通す。

$$
\text{SubLayer} = \text{LayerNorm}(x+\text{SubLayer}(x)), \\
\text{where, SubLayer}(x)\text{ is the function implmented by the sub-layer itself}
$$

```python
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
```


* Decoder
  
Masked Multi-Head Attention, Multi-Head Attention, Feed Forwardの三パーツに分かれる。encoderと同じようにresidual connectionを用いている。

```python
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)
```


### Attention


* Scaled Dot-Product Attention
  
$$
\text{Attention}(Q, K, V) = \text{Softmax}(\frac{QK^{T}}{\sqrt{d_{k}}})V
$$

$$
Q = (\vec{q_{1}}, \vec{q_{2}} ,\dots, \vec{q_{n}})\\
K = (\vec{k_{1}}, \vec{k_{2}}, \dots, \vec{k_{n}})\\
V = (\vec{v_{1}}, \vec{v_{2}}, \dots, \vec{v_{n}})
$$

とすると、Scaled Dot-Product Attentionの出力は$\vec{v_j}$の重み付き和であり、入力$\vec{q_{i}}$に対する$\vec{v_{j}}$の重み$p_{ij}$が、クエリ$\vec{q_{i}}$とキー$\vec{k_{i}}$の類似度をもとに計算される。つまり、 **「$i$番目の処理は$j$番目の情報に注目すべき」ということを「$\vec{q_{i}}$が$\vec{k_{j}}$に似ている」と表現し、$j$番目の情報に注目すべきときは、$\vec{v_{j}}$の重みを重くした重み付き和を出力として返す。**
$QK^{T}$の要素は$q_{i}k_{j}$のように内積になっている。

```python
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn
```

* Multi-Head Attention


$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_{1}, \dots, \text{head}_{h})W^{O}
$$

$$
\text{where head}_{i} = \text{Attention}(QW^{Q}_{i}, KW^{K}_{i}, VW^{V}_{i})
$$

$$
\text{where }W_{i}^{Q} \in \mathbb{R}^{d_{model}\times d_{k}}, 
W_{i}^{K} \in \mathbb{R}^{d_{model}\times d_{k}}, 
W_{i}^{V} \in \mathbb{R}^{d_{model}\times d_{v}}, 
\text{and }W^{O} \in \mathbb{R}^{hd_{v}\times d_{k}}
$$

8層のparallel attention layer, or headsを使用。

encoderのMulti-Head AttentionとdecoderのMasked Multi-Head Attentionは同じものがQ, K, Vとして代入されているように見えるが、$W^{Q}_{l}$、$W^{K}_{l}$、$W^{V}_{l}$を右から掛け、各ベクトルに線形変換を施して回転させることで、どの$\vec{x_{i}}$がどの$\vec{x_{j}}$に注目するかを制御できる。学習を通してこれらの行列を決定することで、入力$\vec{x_{i}}$たちの中で誰が誰に注目するかを制御し、巧みに情報を処理している。
  

Masked Multi-Head Attention: decoderでの出力のために$i$番目の計算には$i$番目までの計算しか使わないようにMaskがなされている。
DecoderのMulti-Head Attention: $Q$はdecoder内で計算された行列。ベクトル列の$i$番目のベクトルは、出力分の$i$番目の単語に対応する。$K$、$V$はencoder内で計算された行列。したがってこの計算は **「$i$番目の出力単語を考えるには、どの入力単語に注目すべきか」をクエリーとキーの類似度を用いて決定し、選ばれた入力単語の意味を表現したバリューベクトル(を回転したもの)の重み付き和を次の処理に渡している。**
```python
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
```

### Position-wise Feed-Forward Networks
2層のlinear-layerとReLUが使われている。512->2048->512次元と変化する。

$$
\text{FFN}(x) = \max(0, \omega W_{1}+b_{i})W_{2}+b_{2}
$$

```python
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
```

### Embeddings and Softmax
linear-layerとソフトマックス関数を通す。ソフトマックス関数の前のlinear-layerは二つのembedding-layerと同じweightを共有する。さらにembedding-layerには$\sqrt{d_{model}}$が掛けられている。  
↓のコードにsoftmaxが無いのは何故だろう？
```python
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
```

### Positional Encoding
文章中のトークンの相対的または絶対的な位置情報を加えるためにPositional Encodingをencoderとdecoderのstackの底？に加える。足し合わせることができるようにembeddingと同じ次元を持つ。以下のように表される。


$$
PE(_{pos, 2i}) = \sin(pos/10000^{2i/d_{model}})
$$

$$
PE(_{pos, 2i+1}) = \cos(pos/10000^{2i/d_{model}})
$$

$$
\text{where }pos \text{ is  the position and } i \text{ is the dimention}
$$


```python
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
```
### full model
```python
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model
```

self-attentionによって計算量を落とすことができる。また、分布を見ることによってモデルが注目している単語の関係が分かるので解釈することができる。

## 4. どうやって有効だと検証した？
* the WMT 2014 English-to-German translation
* the WMT 2014 English-to-French translation
でSOTA  
dropout($P_{drop}=0.1$)とlabel-smoothing($\epsilon_{ls}=0.1$)を使用。

## 5. 議論はある？

## 6. 次に読むべき論文は？
* Neural Machine Translation by Jointly Learning to Align and Translate
* Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation
* Sequence to Sequence Learning with Neural Networks

## 7. 実装
https://github.com/tensorflow/tensor2tensor


### reference
* http://nlp.seas.harvard.edu/annotated-transformer/ 
* 杉山聡. 本質を捉えたデータ分析のための分析モデル入門 統計モデル、深層学習、強化学習等 用途・特徴から原理まで一気通貫!. ソシム, 468p